In [1]:
import torch
embedding = torch.rand((3,2),requires_grad=True)
embedding.retain_grad()

a = torch.rand((2,3),requires_grad=True)
a.retain_grad()

### gradient formulation of the index in topk based selection, proposed in SFI

In [2]:
weight,b = torch.topk(a,k=2,dim=-1)
# b.retain_grad()

c = a.gather(dim=-1,index=b)
c.retain_grad()

# d = torch.zeros(a.size()).scatter(dim=-1,index=b,src=torch.ones(a.size()))
d = embedding[b]*(weight.unsqueeze(dim=-1))

loss = (d**2).sum()
loss.backward()

a.grad

tensor([[1.4690, 1.0891, 0.0000],
        [0.0000, 0.9664, 1.1481]])

### gradient formulation of the index using REINFORCE policy gradient

In [2]:
from torch.distributions import Categorical

rewards = []
policies = []
losses = []

net = torch.nn.Linear(3,3)
distribution = Categorical(net(a).abs())
action = distribution.sample()

In [5]:
res = 
loss = (res**2).sum()

loss.backward()

action, a.grad, embedding.grad, 

(tensor([2, 2]),
 None,
 tensor([[ 0.0000,  0.0000],
         [ 0.0000,  0.0000],
         [ 3.7785, 10.2651]]),
 None)

In [9]:
import torch
from torch.distributions import Normal

a = torch.rand((2,3),requires_grad=True)

dis = Normal(a[0],a[1])
m = dis.rsample()
loss = (m**2).sum()
loss.backward()

a.grad

tensor([[ 0.4937,  1.5606,  2.4589],
        [ 0.0298, -0.3498,  3.3279]])

In [1]:
import torch
import torch.nn as nn
import torch.distributions as distributions


embedding = torch.rand((3,3),requires_grad=True)
embedding.retain_grad()

In [2]:
distribution = torch.tensor([[0.2,0.5,0.3]],requires_grad=True)
distribution.retain_grad()

distribution_hard = nn.functional.softmax(distribution.masked_fill(distribution < distribution.max(), -float('inf')),dim=-1)
ret = distribution - distribution.detach() + distribution_hard

In [4]:
distribution = torch.tensor([[0.2,0.5,0.3]],requires_grad=True)
distribution.retain_grad()

index = distribution.max(-1,keepdim=True)[1]
distribution_hard = torch.zeros_like(distribution, memory_format=torch.legacy_contiguous_format).scatter_(-1, index, 1.0)

ret = distribution - distribution.detach() + distribution_hard

In [6]:
selected = ret.matmul(embedding)
loss = (selected ** 2).sum()
loss.backward()

distribution.grad

tensor([[1.9843, 2.6652, 0.7562]])